In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from datasets import load_dataset

/home/jonathan/powell-phi3-sft/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"
BITS_AND_BYTES_CONFIG = {
    "load_in_4bit": True, 
    "bnb_4bit_quant_type": "nf4", 
    "bnb_4bit_use_double_quant": True, 
    "bnb_4bit_compute_dtype": "float16"
} 


In [3]:
model_id = "microsoft/Phi-3-mini-4k-instruct"

bnb = BitsAndBytesConfig(**BITS_AND_BYTES_CONFIG)

tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
#tok.pad_token = tok.eos_token
# SFT trainer handles padding token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.93s/it]


In [ ]:
lora = LoraConfig(
    init_lora_weights="pissa",
    r=12, 
    lora_alpha=24, 
    lora_dropout=0.1,
    target_modules=["q_proj","k_proj","v_proj","o_proj","up_proj","down_proj","gate_proj"],
    task_type="CAUSAL_LM"
    )

#pissa for training stability + faster convergence
#relatively high dropout to prevent overfitting dataset has 1000 samples
#rank 12 to increase training stability -> due to small dataset

In [5]:
ds = load_dataset("BoostedJonP/JeromePowell-SFT")

Generating train split: 100%|██████████| 1126/1126 [00:00<00:00, 4081.48 examples/s]
